In [1]:
import os
import json
from dotenv import load_dotenv

# 1. 修复导包：统一使用 langchain_core 最新的标准路径
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_core.output_parsers import JsonOutputParser

# 2. 必备起手式：强制加载你本地的 API 密钥
load_dotenv(override=True)

# 3. 实例化大模型 (必须设置 temperature=0，因为 JSON 容不得半点语法错误)
llm = OpenAI(temperature=0)

# =================================================================
# 第一阶段：没有解析器时，大模型随性发挥
# =================================================================
print("【第一阶段：没有解析器的普通请求】")
prompt_normal = PromptTemplate(
    template="List 3 countries in {continent} and their capitals.",
    input_variables=["continent"],
)
print("大模型的回答 (纯文本)：\n", llm.invoke(prompt_normal.format(continent="Asia")).strip())
print("-" * 50)

# =================================================================
# 第二阶段：加入 JSON 解析器，给大模型上“紧箍咒”
# =================================================================
print("【第二阶段：使用 JSON 解析器】")
output_parser = JsonOutputParser()

# 看看 LangChain 是怎么在底层教大模型写 JSON 的
format_instructions = output_parser.get_format_instructions()
print("🤖 解析器自动生成的底层强制指令：\n", format_instructions, "\n")

# 将紧箍咒塞进提示词模板中
prompt_json = PromptTemplate(
    template="List 3 countries in {continent} and their capitals. \n{format_instructions}",
    input_variables=["continent"],
    partial_variables={"format_instructions": format_instructions}
)

final_prompt = prompt_json.format(continent="North America")
print("✉️ 最终发给大模型的完整提示词：\n", final_prompt, "\n")

# 获取大模型的原始回复
output = llm.invoke(final_prompt)
print("📦 大模型的原始回复 (一串纯文本 String):\n", output.strip(), "\n")

# =================================================================
# 第三阶段：见证奇迹，纯文本秒变 Python 对象！
# =================================================================
print("【第三阶段：执行解析】")
countries_data = output_parser.parse(output)

print("✨ 解析后的最终结果 (真正的 Python 字典/列表):")
print(countries_data)
print("类型:", type(countries_data), "\n")

# 用 json.dumps 把它打印得漂漂亮亮
print("📜 格式化后的标准 JSON 字符串:")
print(json.dumps(countries_data, indent=4, ensure_ascii=False))

【第一阶段：没有解析器的普通请求】
大模型的回答 (纯文本)：
 1. Japan - Tokyo
2. India - New Delhi
3. China - Beijing
--------------------------------------------------
【第二阶段：使用 JSON 解析器】
🤖 解析器自动生成的底层强制指令：
 Return a JSON object. 

✉️ 最终发给大模型的完整提示词：
 List 3 countries in North America and their capitals. 
Return a JSON object. 

📦 大模型的原始回复 (一串纯文本 String):
 {
  "Canada": "Ottawa",
  "United States": "Washington D.C.",
  "Mexico": "Mexico City"
} 

【第三阶段：执行解析】
✨ 解析后的最终结果 (真正的 Python 字典/列表):
{'Canada': 'Ottawa', 'United States': 'Washington D.C.', 'Mexico': 'Mexico City'}
类型: <class 'dict'> 

📜 格式化后的标准 JSON 字符串:
{
    "Canada": "Ottawa",
    "United States": "Washington D.C.",
    "Mexico": "Mexico City"
}
